In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('heart_2020_cleaned.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

In [ ]:
df = df.drop('Race', axis=1)
cols_bool = ["HeartDisease", "Smoking", "AlcoholDrinking","Stroke","DiffWalking","PhysicalActivity","Asthma","KidneyDisease","SkinCancer"]

df[cols_bool] = df[cols_bool].replace({'Yes': 1, 'No': 0})
df["Sex"] = df["Sex"].replace({'Male': 1, 'Female': 0})
df["Diabetic"] = df["Diabetic"].map({
    "Yes": 1,
    "No": 0,
    "No, borderline diabetes": 0,
    "Yes (during pregnancy)": 1,
})
df["AgeCategory"] = df["AgeCategory"].map(
    {
    '18-24': 0,
    '25-29': 1,
    '30-34': 2,
    '35-39': 3,
    '40-44': 4,
    '45-49': 5,
    '50-54': 6,
    '55-59': 7,
    '60-64': 8,
    '65-69': 9,
    '70-74': 10,
    '75-79': 11,
    '80 or older': 12
    }
)
df["GenHealth"] = df["GenHealth"].map(
    {
    "Poor": 0,
    "Fair": 1,
    "Good": 2,
    "Very good": 3,
    "Excellent": 4
    }
)

<ipython-input-26-e56100205d3d>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols_bool] = df[cols_bool].replace({'Yes': 1, 'No': 0})
<ipython-input-26-e56100205d3d>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Sex"] = df["Sex"].replace({'Male': 1, 'Female': 0})


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  int64  
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  int64  
 3   AlcoholDrinking   319795 non-null  int64  
 4   Stroke            319795 non-null  int64  
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  int64  
 8   Sex               319795 non-null  int64  
 9   AgeCategory       319795 non-null  int64  
 10  Diabetic          319795 non-null  int64  
 11  PhysicalActivity  319795 non-null  int64  
 12  GenHealth         319795 non-null  int64  
 13  SleepTime         319795 non-null  float64
 14  Asthma            319795 non-null  int64  
 15  KidneyDisease     319795 non-null  int64  
 16  SkinCancer        31

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X = df.drop(columns='HeartDisease')
y = df['HeartDisease']

smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X, y)

df_bal = X_bal.copy()
df_bal['HeartDisease'] = y_bal

In [ ]:
cols_jitter = ['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']

reps = int(1_000_000 / len(df_bal)) + 1

df_expanded = pd.concat([
    df_bal.assign(**{
        col: df_bal[col] + np.random.normal(0, 0.5, size=len(df_bal))
        for col in cols_jitter
    })
    for _ in range(reps)
], ignore_index=True)

df_1M = df_expanded.sample(n=1_000_000, random_state=42).reset_index(drop=True)

In [ ]:
df_1M.to_csv('heart_1M.csv', index=False)